# Entropy (synthetic)

Эксперименты с оценкой энтропии для синтетических данных.

# Преамбула

## Библиотеки

### Tensorflow

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

tfds.disable_progress_bar()
tf.enable_v2_behavior()

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices()

### Math, Numpy, Scipy, Pandas

In [ ]:
import math
import numpy as np
import scipy as sp
import scipy.stats as sps
import scipy.linalg as spl
import pandas as pd

### Matplotlib, Seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Sklearn

In [ ]:
# Деревья.
from sklearn.neighbors import KernelDensity
from sklearn.neighbors import BallTree
from sklearn.neighbors import KDTree

# Метрика.
from sklearn.metrics import pairwise_distances_argmin_min

# Метод главных компонент.
from sklearn.decomposition import PCA

# Выбор модели по кросс-валидации (поиск по сетке).
from sklearn.model_selection import GridSearchCV

### Joblib

In [ ]:
from joblib import Parallel, delayed

n_jobs = 16

### OS, shutil, Json, CSV, copy

In [ ]:
import os
import shutil
import json
import csv
import copy

## Вспомогательное

In [ ]:
# Информация об опыте.
info = dict()

In [ ]:
def normalize_uint8(data, label):
    """Нормализация: `uint8` -> `float32`."""
    return tf.cast(data, tf.float32) / 255.0, label

In [ ]:
def imshow_array(array):
    """Отображение массива нормированных пикселей."""
    plt.axis('off')
    plt.imshow((255.0 * array).astype(np.uint8), cmap=plt.get_cmap("gray"), vmin=0, vmax=255)

In [ ]:
def dataset_Y_to_X(X, Y):
    """Поменять у датасета пары (X, Y) на (X, X) (нужно, например, для обучения автоэнкодера)."""
    return X, X

In [ ]:
def concave_loss(y_true, y_pred):
    """Вогнутая функция потерь, дающая более четкие изображения при обучении."""
    delta = tf.keras.backend.abs(y_true - y_pred)
    squared = tf.keras.backend.square(y_true - y_pred)
    return tf.keras.backend.mean(delta - 0.5 * squared, axis=-1)

## Google Drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## Путь к папке с данными

In [ ]:
#path = "/content/drive/My Drive/Information_v2/"
path = os.path.abspath(os.getcwd()) + "/data/"

# Синтетические данные

Для первоначальных экспериментов данные синтезируются путем сэмплирования точек из некоторого распределения с последующим отображением на некоторое многообразие.

In [ ]:
dataset_dim = 64  # Размерность данных.
latent_dim  = 4  # Реальная (скрытая) размерность данных.
final_noize_stdev = 0.05 # Стандартное отклонение шума, складываемого с выходом функции.
samples_number = 10000 # Размер выборки.
tests_number   = 10000 # Размер тестовой выборки.

In [ ]:
experiments_path = path + "entropy/synthetic/"

In [ ]:
%run Miscellaneous.ipynb

### Выбор случайной величины и отображения

In [ ]:
random_variables = [
    rv_id_ensemble([sps.uniform(0.0, 8.0 * np.pi),
                    sps.uniform(0.0, 8.0 * np.pi),
                    sps.uniform(0.0, 8.0 * np.pi),
                    sps.uniform(0.0, 8.0 * np.pi)])
]

In [ ]:
mappings = [
    mapping_ensemble([mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)]),
                      mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)]),
                      mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)]),
                      mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)])]),
    
    mapping_ensemble([mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)])] * 4),
    
    mapping_ensemble([mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)]),
                      mapping_segmented([mapping_cycloid(2)] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)]),
                      mapping_segmented([mapping_epicycloid(2,5)] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)]),
                      mapping_segmented([mapping_hypocycloid(2,6)] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(8)])]),
    
    mapping_ensemble([mapping_segmented([mapping_epicycloid(3,5),
                                         mapping_hypocycloid(2,6),
                                         mapping_epicycloid(1,1),
                                         mapping_cycloid(1),
                                         mapping_epicycloid(2,2),
                                         mapping_hypocycloid(3,9),
                                         mapping_hypocycloid(1,8),
                                         mapping_cycloid(2)], [(np.pi * i, np.pi * (i+1)) for i in range(8)])] * 4),    
    
    mapping_ensemble([mapping_segmented([mapping_epicycloid(2,5),
                                         mapping_hypocycloid(3,6),
                                         mapping_epicycloid(1,16),
                                         mapping_cycloid(2),
                                         mapping_epicycloid(2,22),
                                         mapping_hypocycloid(1,29),
                                         mapping_hypocycloid(1,8),
                                         mapping_cycloid(3)], [(np.pi * i, np.pi * (i+1)) for i in range(8)])] * 4)
]

In [ ]:
# НОМЕР ИСХОДНОГО РАСПРЕДЕЛЕНИЯ.
# #
# #
random_variable_index = 1
# #
# #

In [ ]:
# НОМЕР ФУНКЦИИ, ЗАДАЮЩЕЙ МНОГООБРАЗИЕ.
# #
# #
mapping_index = 5
# #
# #

In [ ]:
random_variable = random_variables[random_variable_index - 1]
mapping = mappings[mapping_index - 1]

# Проверка входной размерности.
assert latent_dim == mapping.input_dim

true_entropy = random_variable.entropy()

### Генерация набора данных

In [ ]:
# Матрица поворота и повышения размерности.
Q = sps.ortho_group.rvs(dim = dataset_dim)
#Q = np.eye(dataset_dim)
transform = Q[:,:mapping.output_dim]

In [ ]:
def get_samples(random_variable, mapping, samples_number, dataset_dim, transform_matrix, final_noize_stdev = 0.05):
    """
    Генерация набора данных.
    """

    # Данные во внутреннем представлении.
    #np.random.seed(42)
    X = random_variable.rvs(samples_number)
    
    # Отображение шума в пространство большей размерности.
    Y = np.zeros((samples_number, dataset_dim))
    noize = sps.norm(loc=0, scale=final_noize_stdev)
    for i in range(samples_number):
        Y[i] = transform_matrix @ mapping.map(X[i]) + noize.rvs(dataset_dim)
            
    return Y

In [ ]:
samples = get_samples(random_variable, mapping, samples_number, dataset_dim, transform, final_noize_stdev)
tests   = get_samples(random_variable, mapping, tests_number,   dataset_dim, transform, final_noize_stdev)

In [ ]:
projected = np.array([samples[i][0:8] for i in range(1000)])

draw_pair_plot = True
if draw_pair_plot:
    pp = sns.pairplot(pd.DataFrame(projected), height = 2.0, aspect=1.6,
                      plot_kws=dict(edgecolor="k", linewidth=0.0, alpha=0.1, size=0.01, s=0.01),
                      diag_kind="kde", diag_kws=dict(shade=True))

    fig = pp.fig
    fig.subplots_adjust(top=0.93, wspace=0.3)
    t = fig.suptitle('Pairwise Plots', fontsize=14)

### Путь к результатам

In [ ]:
dataset_path = experiments_path + "rv_" + str(random_variable_index) + "_map_" + str(mapping_index) + "/" + str(latent_dim) + "_" + str(dataset_dim) + '/' + ("%.3e" % final_noize_stdev) + "/" + str(samples_number) + "_" + str(tests_number) + "/"

# Оценка энтропии

## Автокодировщик

Сжатие данных предлагается делать автокодировщиком.
Для архитектуры специфицируется только формат входных данных, а также размерность внутреннего представления (кодов).

In [ ]:
# РАЗМЕРНОСТЬ КОДА.
# #
# #
 
codes_dim = 4

# #
# #

In [ ]:
# Число эпох для обучения.
epochs = 10000

In [ ]:
full_path = dataset_path + "autoencoders/" + str(codes_dim) + "_" + str(epochs) + "/"
os.makedirs(full_path, exist_ok=True)

In [ ]:
info['dataset_dim'] = dataset_dim
info['latent_dim'] = latent_dim

info['samples_number'] = samples_number
info['tests_number'] = tests_number

info['codes_dim'] = codes_dim
info['epochs'] = epochs

### Создание модели

In [ ]:
def dense_autoencoder(shape_input, dimension):
    # Инициализация весов.
    init = tf.keras.initializers.RandomNormal(stddev = 0.02)

    # Входные данные генератора / выборки.
    input_layer = tf.keras.layers.Input(shape_input)
    next_layer = input_layer
    next_layer = tf.keras.layers.GaussianNoise(0.02)(next_layer)

    # 1 блок слоёв.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(64, kernel_initializer = init),
                                                  power_iterations = 8)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 2 блок слоёв.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(32, kernel_initializer = init),
                                                  power_iterations = 8)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 3 блок слоёв.
    #next_layer = tf.keras.layers.Dense(8, kernel_initializer = init)(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # Бутылочное горлышко.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(dimension),
                                                  power_iterations = 8)(next_layer)
    bottleneck = tf.keras.layers.Activation('tanh', name='bottleneck')(next_layer)

    # Модель кодировщика.
    encoder = tf.keras.Model(input_layer, bottleneck)

    # Начало модели декодировщика.
    input_code_layer = tf.keras.layers.Input((dimension))
    next_layer = input_code_layer

    # 3 блок слоёв.
    #next_layer = tf.keras.layers.Dense(8, kernel_initializer = init)(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 2 блок слоёв.
    next_layer = tf.keras.layers.Dense(32, kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 1 блок слоёв.
    next_layer = tf.keras.layers.Dense(64, kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 0 блок слоёв.
    next_layer = tf.keras.layers.Dense(shape_input[0])(next_layer) # Подразумевается, что вход - всё равно вектор.
    #next_layer = tf.keras.layers.Activation('tanh')(next_layer)
    
    output_layer = next_layer
    
    # Модель.
    decoder = tf.keras.models.Model(input_code_layer, output_layer) # Декодировщик.
    autoencoder = tf.keras.Sequential([encoder, decoder])

    # Компиляция модели.
    opt = tf.keras.optimizers.Adam(learning_rate = 5e-3)
    autoencoder.compile(loss = 'mse', optimizer = opt, loss_weights = [1.0])
    
    return encoder, decoder, autoencoder

### Загрузка модели

In [ ]:
#encoder = tf.keras.models.load_model(full_path + "encoder.h5")
#decoder = tf.keras.models.load_model(full_path + "decoder.h5")
#autoencoder = autoencoder = tf.keras.Sequential([encoder, decoder])
#autoencoder.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3), loss_weights = [1.0])

#with open(full_path + 'info.json', 'r') as fp:
#    info = json.load(fp)

In [ ]:
encoder, decoder, autoencoder = dense_autoencoder((dataset_dim,), codes_dim)

In [ ]:
history_callback = autoencoder.fit(samples, samples, epochs=epochs, validation_data=(tests, tests), batch_size=samples_number // 10)

In [ ]:
autoencoder.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3), loss_weights = [1.0])

In [ ]:
autoencoder.fit(samples, samples, epochs=1000, validation_data=(tests, tests), batch_size=samples_number)

In [ ]:
# Сохранение динамики loss-функции.
loss_history = np.array(history_callback.history["loss"])
val_loss_history = np.array(history_callback.history["val_loss"])

np.savetxt(full_path + "loss.csv", loss_history, delimiter="\n")
np.savetxt(full_path + "val_loss.csv", val_loss_history, delimiter="\n")

In [ ]:
# Сохранение средних итоговых значений функции потерь.
last_n = 20
last_loss = np.array(loss_history[-last_n:])
last_loss_mean = np.mean(last_loss)
last_loss_std = np.std(last_loss)

last_val_loss = np.array(val_loss_history[-last_n:])
last_val_loss_mean = np.mean(last_val_loss)
last_val_loss_std = np.std(last_val_loss)

info['last_loss_mean'] = last_loss_mean
info['last_loss_std'] = last_loss_std
info['last_val_loss_mean'] = last_val_loss_mean
info['last_val_loss_std'] = last_val_loss_std

with open(dataset_path + "losses.csv", 'a+') as fp:
    writer = csv.writer(fp)
    writer.writerow([codes_dim, last_loss_mean, last_loss_std])

with open(dataset_path + "val_losses.csv", 'a+') as fp:
    writer = csv.writer(fp)
    writer.writerow([codes_dim, last_val_loss_mean, last_val_loss_std])

In [ ]:
# Сохранение моделей.
autoencoder.save(full_path + "autoencoder.h5")
encoder.save(full_path + "encoder.h5")
decoder.save(full_path + "decoder.h5")

In [ ]:
# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

### Получение кодов всех элементов набора данных

In [ ]:
codes = np.array(encoder.predict(samples))

In [ ]:
codes_pca_dim = codes_dim
PCA_codes = PCA(n_components=codes_pca_dim, whiten=True)
codes_pca = np.array(PCA_codes.fit_transform(codes))

### KDE для кодов

In [ ]:
# Загрузка параметров KDE.

#with open(full_path + 'info.json', 'r') as fp:
#    info = json.load(fp)

#kde_codes = KernelDensity(bandwidth=info['bandwidth'], kernel='gaussian')
#kde_codes.fit(codes_pca)

In [ ]:
def smart_gridsearch(begin, end, resolution = 7, rel_x_epsilon = 0.01, rtol = 0.001, n_jobs = 2, cv = 5):
    while True:
        grid = np.logspace(np.log10(begin), np.log10(end), resolution)
        print("Поиск по сетке: ", grid)
        params = {'bandwidth': grid}
        
        grid_search = GridSearchCV(KernelDensity(rtol = rtol), params, n_jobs = n_jobs, verbose = 10, cv = cv)
        grid_search.fit(codes_pca)
        
        if grid_search.best_index_ == 0:
            begin *= begin / end
            end = grid[1]
        elif grid_search.best_index_ == resolution - 1:
            end *= end / begin
            begin = grid[-2]
        else:
            begin = grid[grid_search.best_index_ - 1]
            end = grid[grid_search.best_index_ + 1]

            if end - begin < rel_x_epsilon * grid[grid_search.best_index_]:
                return grid_search 

In [ ]:
KDE_codes = smart_gridsearch(0.05, 0.2, n_jobs = n_jobs).best_estimator_
KDE_codes.set_params(rtol = 0.0)
print(KDE_codes.get_params())

In [ ]:
info['bandwidth'] = KDE_codes.get_params()['bandwidth']

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

with open(dataset_path + "bandwidth.csv", 'a+') as fp:
    writer = csv.writer(fp)
    writer.writerow([codes_dim, info['bandwidth']])

## Подсчёт энтропии

In [ ]:
def _loo_step(bandwidth, samples, i):
    loo_samples = samples
    np.delete(loo_samples, i)
    
    kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
    kde.fit(loo_samples)
    return kde.score_samples([samples[i]])[0]

In [ ]:
def entropy_leave_one_out_parallel(path, bandwidth, samples, n_jobs = 2, first_N = None, parts = 10, recover_saved = False):
    """
    Параллельное вычисление оценки энтропии методом убрать-один-элемент.
    """
    
    # Создание временных папок для сохранения прогресса.
    parts_path = path + "LOO_PARTS/"
    os.makedirs(parts_path, exist_ok=True)

    # Если дано first_N, энтропия будет оцениваться только на первых first_N элементах.
    N = 0
    if first_N is None:
        N = len(samples)
    else:
        N = first_N

    # Число частей и массив, их содержащий.
    N_per_part = N // parts
    log_probs = []

    # Восстанавливаем прогресс, если требуется.
    recovered_parts = 0
    if recover_saved:
        for filename in os.listdir(parts_path):
            if filename.endswith(".csv"):
                log_probs.append(np.loadtxt(parts_path + filename))
                recovered_parts += 1

    print("Восстановлено блоков данных: %d" % recovered_parts)

    # Подсчёт логарифма вероятности в точках.
    for part in range(recovered_parts, parts):
        log_probs.append(
            np.array(
                Parallel(n_jobs = n_jobs, verbose = 10, batch_size = 8)(
                    delayed(_loo_step)(bandwidth, samples, i) for i in range(part * N_per_part, min((part + 1) * N_per_part, N))
                )
            )
        )
        np.savetxt(parts_path + str(part) + ".csv", log_probs[part], delimiter="\n")
    
    # Объединение в один массив.
    log_prob = np.concatenate(log_probs)

    # Суммирование и нахождение стандартного отклонения.
    average = -math.fsum(log_prob) / N    
    squared_deviations = np.zeros(N)
    for i in range(N):
        squared_deviations[i] = (log_prob[i] - average)**2
    standard_deviation = np.sqrt(math.fsum(squared_deviations) / (N * (N - 1)))
    
    # Удаление временных файлов.
    shutil.rmtree(parts_path)
        
    return average, standard_deviation

In [ ]:
latent_entropy, latent_entropy_error = entropy_leave_one_out_parallel(full_path, KDE_codes.get_params()['bandwidth'], codes_pca, n_jobs = n_jobs, recover_saved = False)
print("LH: %f, errLH: %f" % (latent_entropy, latent_entropy_error))

In [ ]:
info['latent_entropy'] = latent_entropy
info['latent_entropy_error'] = latent_entropy_error

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

In [ ]:
with open(dataset_path + "entropy.csv", 'a+') as fp:
    writer = csv.writer(fp)
    writer.writerow([codes_dim, info['latent_entropy'], info['latent_entropy_error']])

In [ ]:
def model_jac(x, model, m, n):
    """
    Вычисление матрицы Якоби модели в точке.
    
    x - точка.
    model - модель.
    m - размерность входа.
    n - размерность выхода.
    """
    
    variables = [tf.Variable([[x[j]]]) for j in range(m)]

    with tf.GradientTape(persistent=True) as t:
        t.watch(variables)
        z = tf.concat(variables, 1)
        f = [model(z)[0][i] for i in range(n)]

    J = np.zeros((m, n))
    for i in range(n):
        for j in range(m):
            J[j][i] = t.gradient(f[i], variables[j]).numpy()

    return J

In [ ]:
def defc_from_jac(J):
    """
    Вычисление коэффициента растяжения для матрицы Якоби.
    """
    
    assert len(J.shape) == 2
    return np.sqrt(np.linalg.det(J @ J.T))

In [ ]:
def _defc_step(sample, model, m, n):
    return np.log(defc_from_jac(model_jac(sample, model, m, n)))

In [ ]:
def transform_entropy_nonparallel(path, samples, model, m, n, first_N = None, parts = 100, recover_saved = False):
    """
    Однопоточное вычисление изменения энтропии под действием декодера.
    """
    
    # Создание временных папок для сохранения прогресса.
    parts_path = path + "TE_PARTS/"
    os.makedirs(parts_path, exist_ok=True)

    # Если дано first_N, энтропия будет оцениваться только на первых first_N элементах.
    N = 0
    if first_N is None:
        N = len(samples)
    else:
        N = first_N

    # Число частей и массив, их содержащий.
    N_per_part = N // parts
    log_defcs = []

    # Восстанавливаем прогресс, если требуется.
    recovered_parts = 0
    if recover_saved:
        for filename in os.listdir(parts_path):
            if filename.endswith(".csv"):
                log_defcs.append(np.loadtxt(parts_path + filename))
                recovered_parts += 1

    print("Восстановлено блоков данных: %d" % recovered_parts)

    # Подсчёт логарифма вероятности в точках.
    for part in range(recovered_parts, parts):
        print("Новый блок: %d" % part)
        log_defcs.append(
            np.array([_defc_step(samples[i], model, m, n) for i in range(part * N_per_part, min((part + 1) * N_per_part, N))])
            )
        np.savetxt(parts_path + str(part) + ".csv", log_defcs[part], delimiter="\n")
    
    # Объединение в один массив.
    log_defc = np.concatenate(log_defcs)

    # Суммирование и нахождение стандартного отклонения.
    average = math.fsum(log_defc) / N    
    squared_deviations = np.zeros(N)
    for i in range(N):
        squared_deviations[i] = (log_defc[i] - average)**2
    standard_deviation = np.sqrt(math.fsum(squared_deviations) / (N * (N - 1)))
    
    # Удаление временных файлов.
    shutil.rmtree(parts_path)
        
    return average, standard_deviation

In [ ]:
transform_entropy, transform_entropy_error = transform_entropy_nonparallel(full_path, codes, decoder, codes_dim, dataset_dim, first_N = 2000, recover_saved = False)
print("TH: %f, errTH: %f" % (transform_entropy, transform_entropy_error))

In [ ]:
info['transform_entropy'] = transform_entropy
info['transform_entropy_error'] = transform_entropy_error

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

In [ ]:
with open(dataset_path + "entropy.csv", 'a+') as fp:
    writer = csv.writer(fp)
    writer.writerow([codes_dim, info['transform_entropy'], info['transform_entropy_error']])

In [ ]:
# Коэффициент растяжения при денормализации.
PCA_codes_defc = np.abs(np.linalg.det( PCA_codes.inverse_transform(np.eye(codes_pca_dim)) -
                                       PCA_codes.inverse_transform(np.zeros((codes_pca_dim, codes_pca_dim))) ))

In [ ]:
# Соответствующая энтропия.
PCA_codes_transform_entropy = np.log(PCA_codes_defc)

print("PCA_TH: %f" % (PCA_codes_transform_entropy))

In [ ]:
info['PCA_codes_transform_entropy'] = PCA_codes_transform_entropy

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

In [ ]:
# Итоговая оценка энтропии.
entropy = latent_entropy + transform_entropy + PCA_codes_transform_entropy
entropy_error = latent_entropy_error + transform_entropy_error

print("H: %f, errH: %f" % (entropy, entropy_error))

In [ ]:
info['entropy'] = entropy
info['entropy_error'] = entropy_error

info['true_entropy'] = true_entropy
info['true_error'] = np.abs(entropy - true_entropy)

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

In [ ]:
print("Итоговый результат:")
print("Оценка: %f\nИстинное значение: %f\nОшибка: %.3e" % (entropy, true_entropy, entropy - true_entropy))